### Search Engine with Tools and Agents

In [4]:
## Arxiv--Research
## Tool creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [6]:
# used inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [12]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [13]:
tools = [wiki,arxiv]

In [15]:
## custom tools [RAG Tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [19]:
embedding=(OllamaEmbeddings(model="mxbai-embed-large"))  ##by default it ues llama2

In [20]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,embedding=embedding)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002606D23EA90>, search_kwargs={})

In [24]:
from langchain_core.tools import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith")
retriever_tool.name

'langsmith-search'

In [25]:
tools = [wiki,arxiv,retriever_tool]

In [26]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Generative AI\\projects\\Tools&Agents\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 StructuredTool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x00000260A56C14E0>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x000002606C087560>)]

In [31]:
## Run all this tools with Agents and LLM Models
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import os
from getpass import getpass

groq_api_key = os.getenv('GROQ_API_KEY') or os.environ.get('GROQ_API_KEY')
if not groq_api_key:
    groq_api_key = getpass("Enter your GROQ_API_KEY (input hidden): ")

if not groq_api_key:
    raise EnvironmentError("GROQ_API_KEY not found. Set the GROQ_API_KEY environment variable or add it to a .env file.")

llm = ChatGroq(groq_api_key=groq_api_key, model="meta-llama/llama-4-scout-17b-16e-instruct")

In [36]:
## prompt template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are an helpful AI assistant
"""),
    ("human", "{input}"),
])


In [37]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nYou are an helpful AI assistant\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]

In [53]:
from langchain.agents import create_agent
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are an helpful AI assistant"
)


In [56]:
agent.invoke({"input":"Tell me about Langsmith"})   

BadRequestError: Error code: 400 - {'error': {'message': "tool call validation failed: parameters for tool wikipedia did not match schema: errors: [missing properties: 'query']", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': 'I understand the guidelines. I\'m ready to assist you with the functions provided:\n\n[\n    {"name":"wikipedia","description":"A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.","parameters":{"properties":{"query":{"description":"query to look up on wikipedia","type":"string"}},"required":["query"],"type":"object"}},\n    {"name":"arxiv","description":"A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on ar